In [5]:
!pip install imbalanced-learn

SyntaxError: invalid syntax (1990127073.py, line 2)

In [5]:
# models/models_ran.ipynb

# 📌 שלב 1: ייבוא ספריות
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

# 📌 שלב 2: הגדרות בסיס
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid', palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)

# 📌 שלב 3: קריאת הקובץ המעודכן ביותר
notebook_path = os.getcwd() 
excel_path = os.path.join(notebook_path,"..", 
 "Excel_Updates","Update 52 - English Translation","merged_text_features_filled_T3.xlsx"
)

# בדיקה מה יש בגליונות
xls = pd.ExcelFile(excel_path)
print("Available sheets:", xls.sheet_names)

# 📌 שלב 4: טען את הגיליון שתרצה (לדוגמה - גיליון ראשון)
df = pd.read_excel(xls, sheet_name=0)  # אפשר גם sheet_name="שם_גיליון"

# תצוגה ראשונית של הנתונים
print("Shape:", df.shape)
df.head()


Available sheets: ['Sheet1']
Shape: (51, 44)


,Random patient number,Random hospitalization case number,female,Hospitalization age,Marital status_divorced,Marital status_married,Marital status_single,Referral factor category_hospital,Referring factor category_Legal and security factors,Referral agency category_Other medical institution,Referring factor category_self-referral,Referral factor category_District psychiatrist,Category referred for clinical treatment,Category referred to_institutions,Category: Referred to a psychiatric clinic,Category: Referred to_treating_doctor,Category redirected to_empty,Legal Status Category_Consent Hospitalization,Legal Status Category_Involuntary Hospitalization,Legal Status Category_Urgent Involuntary Hospitalization,Legal Status Category_Legal Entities,Region of origin_Ethiopia,Area of Origin_Soviet Union,Origin_Israel,Age of Aliya,Returned to hospital within a month,Returned to hospital within 3 months,Month_January,Month_February,Month_March,Month_May,Month_June,Month_July,Month_August,Month_September,Month_October,Month_November,Month_December,Year_2021,Year_2022,Year_2023,Year_2024,Receipt text,Release text
0,4.0,47098.0,0.0,0.263158,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.21519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"Admission to the emergency room: 38 years old,...",Release report:
1,24.0,77161.0,0.0,0.473684,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Admission to the emergency room: 54 years old,...",Release report:
2,24.0,81610.0,0.0,0.486842,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Emergency room admission: 55 years old, well k...",Release report:
3,24.0,15554.0,0.0,0.486842,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Admission to the emergency room: 55 years old,...",Release report:
4,24.0,24408.0,0.0,0.500000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"Admission to the emergency room: Born in 1965,...",Release report:


In [6]:
df["Returned to hospital within a month"].value_counts(normalize=True)


Returned to hospital within a month
0.0     0.78
1.0     0.20
10.0    0.02
Name: proportion, dtype: float64

In [7]:
df["Returned to hospital within 3 months"].value_counts(normalize=True)


Returned to hospital within 3 months
0.0     0.58
1.0     0.40
20.0    0.02
Name: proportion, dtype: float64

In [8]:
!pip install transformers



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install torch torchvision torchaudio



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
#בגלל שיש לנו חוסר איזון בעמודה חזר לאשפוז תוך חודש-יש הרבה יותר מטופלים שלא חזרו מאשר חזרו, אז המודל יכול לעשות הרבה טעויות, לכן אנחנו נרצה למשקל את הנתונים שבעמודה הזו, כי ברגע שיש לנו יותר מדי כאלו שלא חזרו לאשפוז אז ברט יחזה לרוב שהמטופל לא יחזור. זה בעצם הטייה 
#שימוש בweighted loss 
#שימוש בBERT
#שימוש בברט לחיזוי חזרה לאשפוז תוך חודש
# שילוב טקסט קבלה + שחרור לשדה אחד
df["full_text"] = df["Receipt text"].fillna('') + " " + df["Release text"].fillna('')

# מטרה: חיזוי חזרה לאשפוז תוך חודש

df = df[(df["full_text"].str.strip() != "") & (df["Returned to hospital within a month"].notnull())]
df["label"] = df["Returned to hospital within a month"].astype(int)

# טוקניזציה
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [11]:
print(df.columns.tolist())


['Random patient number', 'Random hospitalization case number', 'female', 'Hospitalization age', 'Marital status_divorced', 'Marital status_married', 'Marital status_single', 'Referral factor category_hospital', 'Referring factor category_Legal and security factors', 'Referral agency category_Other medical institution', 'Referring factor category_self-referral', 'Referral factor category_District psychiatrist', 'Category referred for clinical treatment', 'Category referred to_institutions', 'Category: Referred to a psychiatric clinic', 'Category: Referred to_treating_doctor', 'Category redirected to_empty', 'Legal Status Category_Consent Hospitalization', 'Legal Status Category_Involuntary Hospitalization', 'Legal Status Category_Urgent Involuntary Hospitalization', 'Legal Status Category_Legal Entities', 'Region of origin_Ethiopia', 'Area of Origin_Soviet Union', 'Origin_Israel', 'Age of Aliya', 'Returned to hospital within a month', 'Returned to hospital within 3 months', 'Month_Janu

In [57]:
#שלב 6: הכנת קלאס דאטאסט ו־DataLoader
from torch.utils.data import Dataset, DataLoader
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


In [58]:
#שלב 7: פיצול נתונים וחישוב class weights
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train, X_test, y_train, y_test = train_test_split(
    df["full_text"].tolist(), 
    df["label"].tolist(), 
    test_size=0.2, 
    stratify=df["label"],
    random_state=42
)

# Class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# יצירת דאטהסט
train_dataset = TextDataset(X_train, y_train, tokenizer)
test_dataset = TextDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [14]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu



Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
#שלב 8
from transformers import BertForSequenceClassification
import torch.nn as nn
import torch

class WeightedBERTModel(nn.Module):
    def __init__(self, class_weights):
        super().__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        self.loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, **inputs):
        # נסנן את הפרמטרים שהמודל תומך בהם (input_ids, attention_mask, token_type_ids, labels)
        bert_inputs = {}
        for key in ['input_ids', 'attention_mask', 'token_type_ids', 'labels']:
            if key in inputs:
                bert_inputs[key] = inputs[key]

        output = self.bert(**bert_inputs)
        logits = output.logits
        loss = None
        if 'labels' in bert_inputs:
            loss = self.loss_fn(logits, bert_inputs['labels'])
        return {'loss': loss, 'logits': logits}

# הנחה: device ו-class_weights כבר הוגדרו
model = WeightedBERTModel(class_weights.to(device)).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
!pip install ipywidgets



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [18]:
!pip install ipywidgets
!jupyter nbextension install --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py widgetsnbextension --sys-prefix



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nb

In [19]:
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [20]:
from tqdm import tqdm


In [21]:
# שלב 9: הערכה (Evaluation) של המודל המאומן
from tqdm import tqdm
epochs = 3

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("🚀 Starting Training...\n")

model.train()
for epoch in range(epochs):
    total_loss = 0
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output['loss']
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")



🚀 Starting Training...


🔁 Epoch 1/3


Epoch 1:  20%|████████████▌                                                  | 1/5 [01:28<05:54, 88.71s/it, loss=0.679]

Step 0, Batch loss: 0.6790


Epoch 1: 100%|███████████████████████████████████████████████████████████████| 5/5 [08:14<00:00, 98.86s/it, loss=0.773]



📉 Average training loss for epoch 1: 0.7233

🔁 Epoch 2/3


...


📉 Average training loss for epoch 2: 0.7081

🔁 Epoch 3/3


Epoch 3:  20%|████████████▌                                                  | 1/5 [01:22<05:31, 82.79s/it, loss=0.649]

Step 0, Batch loss: 0.6486


Epoch 3: 100%|████████████████████████████████████████████████████████████████| 5/5 [06:41<00:00, 80.35s/it, loss=0.85]


📉 Average training loss for epoch 3: 0.7063


In [53]:
# שלב 9: הערכה (Evaluation) של המודל המאומן
from tqdm import tqdm
epochs = 3

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("🚀 Starting Training...\n")

model.train()
for epoch in range(epochs):
    total_loss = 0
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output['loss']
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")



🚀 Starting Training...


🔁 Epoch 1/5


  0%|                                                                                            | 0/5 [00:48<?, ?it/s]


IndexError: Target -9223372036854775808 is out of bounds.

In [22]:
#בגלל שתוצאות ההרצה לא משהו אני בודקת אם המשקלים מחושבים סבבה
print("Class weights:", class_weights)


Class weights: tensor([0.6290, 2.4375])


In [46]:
print(df['Returned to hospital within a month'].unique())


[ 0.  1. nan 10.]


In [47]:
print(df['Returned to hospital within a month'].isna().sum())


1


In [48]:
# מסירה שורות עם ערכים חסרים או לא חוקיים בעמודה
df = df[df['Returned to hospital within a month'].isin([0.0, 1.0])]

# המרה לסוג int
df['label'] = df['Returned to hospital within a month'].astype(int)


In [49]:
print(df['Returned to hospital within a month'].unique())

[0. 1.]


In [50]:
print(df['Returned to hospital within a month'].value_counts(dropna=False))
print(df['Returned to hospital within a month'].isna().sum())


Returned to hospital within a month
0.0    39
1.0    10
Name: count, dtype: int64
0


In [51]:
# מסירה שורות עם ערכים חסרים או לא חוקיים בעמודה
df = df[df['Returned to hospital within a month'].isin([0.0, 1.0])]

# המרה לסוג int
df['label'] = df['Returned to hospital within a month'].astype(int)

from transformers import BertTokenizer, get_scheduler
from torch.optim import AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader, random_split
import torch
from tqdm import tqdm
import pandas as pd
import os

# 💾 טעינת הדאטה
notebook_path = os.getcwd()
excel_path = os.path.join(notebook_path, "..", "Excel_Updates", "Update 52 - English Translation", "merged_text_features_filled_T3.xlsx")
df = pd.read_excel(excel_path)

# אין צורך במיפוי אם הערכים כבר 0/1
df['label'] = df['Returned to hospital within a month'].astype(int)

# שילוב טקסטים
df["full_text"] = df["Receipt text"].fillna('') + " " + df["Release text"].fillna('')

# ✂️ טוקניזציה
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(
    df["full_text"].tolist(),
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# הוספת תוויות
encodings['labels'] = torch.tensor(df['label'].tolist()).long()

# 📦 הגדרת Dataset
class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.encodings.items()}
    def __len__(self):
        return len(self.encodings["input_ids"])

dataset = CustomDataset(encodings)

# ✂️ חלוקה ל־Train ו־Validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 🔄 DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# ⚙️ הגדרות אימון
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ← הנח שמודל BERT מותאם קיים
model = model.to(device)

epochs = 7
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# 🚀 התחלת אימון
print("🚀 Starting Training...\n")

for epoch in range(epochs):
    model.train()
    total_loss = 0
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, leave=True)

    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output['loss']
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")

    # ------------------ 📊 הערכת ביצועים ------------------
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch['labels']
            output = model(**batch)
            logits = output['logits']
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"\n📊 Evaluation Metrics (Epoch {epoch+1}):")
    print(f"  Accuracy:  {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [52]:
# שלב 9: הערכה (Evaluation) של המודל המאומן
from tqdm import tqdm
epochs = 7

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

print("🚀 Starting Training...\n")

model.train()
for epoch in range(epochs):
    total_loss = 0
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        loss = output['loss']
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")



🚀 Starting Training...


🔁 Epoch 1/7


  0%|                                                                                            | 0/5 [00:39<?, ?it/s]


IndexError: Target -9223372036854775808 is out of bounds.

In [56]:

#NLP לעמודת 3 חודשים
#חלק 1: הכנת הטקסטים והתגיות עבור "תוך 3 חודשים
# שילוב טקסט קבלה + שחרור לשדה אחד עבור חיזוי תוך 3 חודשים
df_3months = df[(df["full_text"].str.strip() != "") & (df["Returned to hospital within 3 months"].notnull())].copy()
df_3months["label"] = df_3months["Returned to hospital within 3 months"].astype(int)


In [59]:
# חלק 2: פיצול נתונים + חישוב משקלים
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

X_train_3m, X_test_3m, y_train_3m, y_test_3m = train_test_split(
    df_3months["full_text"].tolist(),
    df_3months["label"].tolist(),
    test_size=0.2,
    stratify=df_3months["label"],
    random_state=42
)

# חישוב משקלי מחלקות
class_weights_3m = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_3m), y=y_train_3m)
class_weights_3m = torch.tensor(class_weights_3m, dtype=torch.float).to(device)


In [60]:
#חלק 3: יצירת דאטהסטים ו־DataLoader
train_dataset_3m = TextDataset(X_train_3m, y_train_3m, tokenizer)
test_dataset_3m = TextDataset(X_test_3m, y_test_3m, tokenizer)

train_loader_3m = DataLoader(train_dataset_3m, batch_size=8, shuffle=True)
test_loader_3m = DataLoader(test_dataset_3m, batch_size=8)


In [61]:
#חלק 4: יצירת מודל חדש ל־3 חודשים
model_3m = WeightedBERTModel(class_weights_3m).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
#חלק 5: אימון מודל חיזוי תוך 3 חודשים

from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

epochs = 3
optimizer_3m = AdamW(model_3m.parameters(), lr=5e-5)

num_training_steps_3m = epochs * len(train_loader_3m)
lr_scheduler_3m = get_scheduler(
    name="linear", optimizer=optimizer_3m, num_warmup_steps=0, num_training_steps=num_training_steps_3m
)

print("🚀 Starting Training for 3-Month Readmission...\n")

model_3m.train()
for epoch in range(epochs):
    total_loss = 0
    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader_3m, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model_3m(**batch)
        loss = output['loss']
        loss.backward()
        optimizer_3m.step()
        lr_scheduler_3m.step()
        optimizer_3m.zero_grad()

        total_loss += loss.item()

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader_3m)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")


🚀 Starting Training for 3-Month Readmission...


🔁 Epoch 1/3


Epoch 1:  20%|████████████▍                                                 | 1/5 [01:42<06:50, 102.70s/it, loss=0.834]

Step 0, Batch loss: 0.8340


Epoch 1: 100%|███████████████████████████████████████████████████████████████| 5/5 [06:56<00:00, 83.27s/it, loss=0.649]



📉 Average training loss for epoch 1: 0.7547

🔁 Epoch 2/3


...


📉 Average training loss for epoch 2: 0.6886

🔁 Epoch 3/3


Epoch 3:  20%|████████████▌                                                  | 1/5 [01:05<04:20, 65.03s/it, loss=0.625]

Step 0, Batch loss: 0.6245


Epoch 3: 100%|████████████████████████████████████████████████████████████████| 5/5 [04:44<00:00, 56.84s/it, loss=0.72]


📉 Average training loss for epoch 3: 0.6741


In [64]:
from sklearn.metrics import accuracy_score

print("🚀 Starting Training for 3-Month Readmission...\n")

model_3m.train()
for epoch in range(epochs):
    total_loss = 0
    all_preds = []
    all_labels = []

    print(f"\n🔁 Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader_3m, leave=True)
    for step, batch in enumerate(loop):
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch['labels']

        output = model_3m(**batch)
        loss = output['loss']
        logits = output['logits']

        loss.backward()
        optimizer_3m.step()
        lr_scheduler_3m.step()
        optimizer_3m.zero_grad()

        total_loss += loss.item()

        # חישוב תחזיות
        preds = logits.argmax(dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

        if step % 100 == 0:
            print(f"Step {step}, Batch loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader_3m)
    epoch_accuracy = accuracy_score(all_labels, all_preds)
    print(f"\n📉 Average training loss for epoch {epoch+1}: {avg_loss:.4f}")
    print(f"✅ Training accuracy for epoch {epoch+1}: {epoch_accuracy:.4f}")


🚀 Starting Training for 3-Month Readmission...


🔁 Epoch 1/3


Epoch 1:  20%|████████████▌                                                  | 1/5 [01:09<04:36, 69.21s/it, loss=0.642]

Step 0, Batch loss: 0.6416


Epoch 1: 100%|███████████████████████████████████████████████████████████████| 5/5 [05:09<00:00, 61.90s/it, loss=0.637]



📉 Average training loss for epoch 1: 0.6407
✅ Training accuracy for epoch 1: 0.7436

🔁 Epoch 2/3


...


📉 Average training loss for epoch 2: 0.6409
✅ Training accuracy for epoch 2: 0.7436

🔁 Epoch 3/3


Epoch 3:  20%|████████████▌                                                  | 1/5 [01:28<05:52, 88.09s/it, loss=0.571]

Step 0, Batch loss: 0.5710


Epoch 3: 100%|███████████████████████████████████████████████████████████| 5/5 [5:53:21<00:00, 4240.29s/it, loss=0.653]



📉 Average training loss for epoch 3: 0.6412
✅ Training accuracy for epoch 3: 0.7436
